# Login Authentication Mock Data

Notebook de desenvolvimento responável por realizar processo de mock dos dados e evenviar os dados para o eventhub utilizando autenticação com SPN


<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://raw.githubusercontent.com/Foiac/MobileFraudDetectSolution/main/Editaveis/Imagens/login-authetication-mock-data.png" alt="SparkStreaming Ingest" style="width: 800px">
</div>

#### Import dependecies

In [23]:
import random
import json
from faker import Faker
from datetime import datetime
from azure.eventhub import EventHubProducerClient, EventData
from azure.identity import ClientSecretCredential
import pandas as pd
import threading
from dotenv import load_dotenv
import os

#### Path definitions and secrets

In [24]:
load_dotenv()

azure_tenant_id: str = os.environ["AZURE_TENANT"]
azure_client_id: str = os.environ["CLIENT_ID"]
azure_client_secret: str = os.environ["CLIENT_SECRET"] 
eventhub_namespace: str = os.environ["EVENTHUB_NAMESPACE"]
eventhub_namespace_fully: str = f"{eventhub_namespace}.servicebus.windows.net"
eventhub_name: str = os.environ["EVENTHUB_NAME"]

In [25]:
file_path = 'phonedataset.csv'

df = pd.read_csv(file_path, sep=';')
phone_list = df.values.tolist()

with open('city_clusters.json', 'r', encoding='utf-8') as file:
    city_clusters = json.load(file)

#### Initial Definitions

In [26]:
start_date = datetime(2024, 11, 1, 0)
end_date = datetime(2024, 11, 2, 23)

In [42]:
fake = Faker()

transaction_prob_true = 0.78
transaction_prob_false = 0.22
transaction_prob = [transaction_prob_true, transaction_prob_false] 

app_version_prob_v201 = 0.05
app_version_prob_v202 = 0.20
app_version_prob_v203 = 0.75
app_version_prob = [app_version_prob_v201, app_version_prob_v202, app_version_prob_v203] 

error_prob_success = 0.90
error_prob_incorrect_pass = 0.08
error_prob_user_not_fount = 0.02
error_prob = [error_prob_success, error_prob_incorrect_pass, error_prob_user_not_fount] 

uids = [str(fake.random_number(digits=11, fix_len=True)) for _ in range(2)]

#### Function to Create SPN Client 

In [43]:
def spn_authentication():
    """
    Creates an Event Hub producer client using SPN credentials.

    Returns:
        EventHubProducerClient: Client for sending events to Azure Event Hub.
    """
    
    print("init procces")
    credential = ClientSecretCredential(tenant_id=azure_tenant_id, 
                                        client_id=azure_client_id,
                                        client_secret=azure_client_secret)
    print("credential create")
    producer = EventHubProducerClient(fully_qualified_namespace=eventhub_namespace_fully,
                                        eventhub_name=eventhub_name,
                                        credential=credential)
    print("producer create")

    return producer

#### Function to Generate Random Data

In [48]:
def generate_random_data():
    """
    Generates a dictionary representing random simulated data for login authentication events.

    This function creates data for testing or simulation purposes, including fields like
    geographic location, device information, network details, and authentication attributes.

    Returns:
        dict: A dictionary containing randomly generated data fields:
            - imei (str): Randomly generated 15-digit IMEI number.
            - mac (str): Randomly generated MAC address.
            - network (str): Randomly chosen network provider (e.g., "VIVO", "TIM").
            - client_ip (str): Random IPv4 address.
            - latitude (float): Latitude based on a random city cluster and Gaussian noise.
            - longitude (float): Longitude based on a random city cluster and Gaussian noise.
            - uid (str): Randomly generated user ID (UID) or chosen from a predefined list.
            - password (str): Random password with a mix of digits, upper- and lower-case letters.
            - transaction (str): Simulated transaction status ("true" or "false").
            - api (str): API name for the simulated event ("login-authentication").
            - endpoint (str): API endpoint ("v1/login").
            - os (str): Randomly selected operational version from the phone list.
            - phone_brand (str): Phone brand associated with the operational version.
            - app_version (str): Randomly chosen application version.
            - error (str): Simulated error code (e.g., "0", "INCORRECT_PASS").
            - timestamp (str): Random UNIX timestamp in milliseconds.
            """

    city = random.choice(city_clusters)
    latitude = random.gauss(city["lat"], city["std_dev"])
    longitude = random.gauss(city["long"], city["std_dev"])

    brand_version = random.choice(phone_list)
    phone_brand = brand_version[0]
    operational_version = brand_version[1]

    fraud_prob = 0.15

    uid = random.choice(uids) if random.random() < fraud_prob else str(fake.random_number(digits=11, fix_len=True))

    return {
        "imei": str(fake.random_number(digits=15, fix_len=True)),
        "mac": ':'.join([f"{random.randint(0, 255):02X}" for _ in range(6)]),
        "network": random.choice(["VIVO", "TIM", "CLARO", "OI"]),
        "client_ip": fake.ipv4(),
        "latitude": latitude,
        "longitude": longitude,
        "uid": uid,
        "password": fake.password(length=6, special_chars=False, digits=True, upper_case=True, lower_case=True),
        "transaction": random.choices(["true", "false"], weights=transaction_prob, k=1)[0],
        "api": "login-authentication",
        "endpoint": "v1/login",
        "os": operational_version,
        "phone_brand": phone_brand,
        "app_version": random.choices(["2.0.1", "2.0.2", "2.0.3"], weights=app_version_prob, k=1)[0],
        "error": random.choices(["0", "INCORRECT_PASS", "USER_NOT_FOUND"], weights=error_prob, k=1)[0],
        "timestamp": str(fake.unix_time(start_datetime=start_date, end_datetime=end_date) * 1000)
    }

#### Function to Send Data do Eventhub

In [45]:
def send_message_to_eventhub(eh_producer, msg_number, trh_number):
    """
    Sends a specified number of JSON messages to Azure Event Hub using the provided producer client.

    Args:
        eh_producer (EventHubProducerClient): Event Hub producer client.
        msg_number (int): Number of messages to send.
        trh_number (int): Transaction identifier for logging.

    Raises:
        ValueError: If a message exceeds batch size limits.
        Exception: For unexpected errors during sending.
    """
        
    for i in range(msg_number):

        message = json.dumps(generate_random_data())

        try:
            with eh_producer:
                event_data_batch = eh_producer.create_batch()

                event_data_batch.add(EventData(message))

                eh_producer.send_batch(event_data_batch)
                
                print(f"[{trh_number}] - Mensagem enviada com sucesso! - {message}")
        except ValueError as ve:
            print(f"Erro ao enviar mensagem: {ve}")
        except Exception as e:
            print(f"Erro inesperado ao enviar mensagem: {e}")
        finally:
            eh_producer.close()


#### Start Thread to Send Data

In [ ]:
eh_producer = spn_authentication()

msg_number = 200

thread_1 = threading.Thread(target=send_message_to_eventhub, args = (eh_producer, msg_number, 1))
thread_2 = threading.Thread(target=send_message_to_eventhub, args = (eh_producer, msg_number, 2))

thread_1.start()
thread_2.start()

thread_1.join()
thread_2.join()